In [17]:
import re

import dotenv
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

import os
from dotenv import load_dotenv



In [18]:
# get the API access keys from file .env

load_dotenv()  # take environment variables from .env.
config = dotenv.dotenv_values(".env")   # config = {"USER": "foo", "EMAIL": "foo@example.org"}

print(config)

# keys and tokens from the Twitter Dev Console
consumer_key = config.get('consumer_key')
consumer_secret = config.get('consumer_secret')
access_token = config.get('access_token')
access_token_secret = config.get('access_token_secret')


OrderedDict([('consumer_key', '0r3gTc4w2OCnUcimnZsejmzXm'), ('consumer_secret', 'cjOHhBoQH5rGFfhDmpt5oS8FegYXuVOKjlCBm5ZRJyWn2AktSg'), ('access_token', '29489350-HqS8DnyFO0ccw37DxQ9HsyXKvJgC2rbxYqmthGCDG'), ('access_token_secret', 'EOYSNpR7gSIzdVHO4NgPGAQeObT1rllxh05E4gNik2uXg')])


# create a twitter client class


In [19]:
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''

        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
  
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
  
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
  
    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
  
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search_tweets(q = query, count = count)
  
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
  
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
  
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
  
            # return parsed tweets
            return tweets
  
        except tweepy.TweepyException as e:
            # print error (if any)
            print("Error : " + str(e))


# create the main and call 

In [20]:
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query = '$TSLA', count = 200)
  
    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
    # percentage of neutral tweets
    print("Neutral tweets percentage: {} % \
        ".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets)))
  
    # printing first 5 positive tweets
    print("\n\nPositive tweets:")
    for tweet in ptweets[:10]:
        print(tweet['text'])
  
    # printing first 5 negative tweets
    print("\n\nNegative tweets:")
    for tweet in ntweets[:10]:
        print(tweet['text'])
  
if __name__ == "__main__":
    # calling main function
    main()

Positive tweets percentage: 47.05882352941177 %
Negative tweets percentage: 14.117647058823529 %
Neutral tweets percentage: 38.8235294117647 %         


Positive tweets:
$DJI Trade idea💡

Great risk/reward here. 

$DKNG $SPY $QQQ $IWM $SLV $MGM $ROKU $AAPL $AMZN $TSLA https://t.co/0qHNbqJ5ye
RT @EliBurton_: I give you the hypocrisy of the main stream media. $twtr $tsla
RT @ProblemSniper: Exciting week ahead: $NFLX $TSLA $LRCX $SNAP $IBM $ASML $AXP https://t.co/W93Tl2lkWc
RT @SamAntar: Crazy Eddie Memoirs: The distraction is more effective than the lie. $TSLA $TWTR
Highest success rate and get instant help and updates and free trials join us 

$VST, $TSLA, $IRNT, $AAPL, $BBIG,… https://t.co/4p7Vzs7aYi
@TSLAFanMtl She definitely sounded very bullish in her latest ITK.

PT is a bit too aggressive to me but who knows!… https://t.co/TVvxj36fub
Great place to learn trading and to get instant help join our community 

$SPY $NDX $IWM $QQQ $BTC $ETH $AAPL $TSLA… https://t.co/9qrenwhqan
Follow 